In [14]:
%reset-f

In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

import warnings
warnings.filterwarnings("always",category=UserWarning)
from res_ind_lib import *
import os, time



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#Folder where outputs are stored
out_folder = "scorecards/"
os.makedirs(out_folder,exist_ok=True)

In [17]:
debug=False
concat_sensit = True


# Functions

In [18]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
def make_excel_report(derivatives, filename ="signs.xlsx" ):
    writer= pd.ExcelWriter(filename, engine='xlsxwriter')
    workbook=writer.book
    # Add a format. Light red fill with dark red text.
    red = workbook.add_format({'bg_color': '#FFC7CE',
                                   'font_color': '#9C0006'})

    blue = workbook.add_format({'bg_color': '#92c5de',
                                   'font_color': '#000061'})
    for outname in score_card_set:
    #for outname in ["resilience"]:
        derivatives[outname].to_excel(writer,sheet_name=outname)
        writer.sheets[outname].conditional_format('B2:BB600', {'type':'cell',
                                        'criteria': '>',
                                        'value':    0,
                                        'format':   blue})
        writer.sheets[outname].conditional_format('B2:BB600', {'type':'cell',
                                        'criteria': '<',
                                        'value':    0,
                                        'format':   red})
        writer.sheets[outname].freeze_panes(1, 1)

    return writer

# Data

In [19]:
df_original=pd.read_csv("results/df_original.csv",index_col="country").dropna()

#appends "countries" from the sensitivity analysis
df_sensit=pd.read_csv("results/df_sensit_input.csv",index_col="country").dropna()

#df_original=df_original.append(df_sensit)


df_with_results=pd.read_csv("results/df.csv",index_col="country")

#Rankings (how the country sits with respect to all other in the sample, for each input. not used)
ranks = df_with_results.dropna(how="all",axis=1).dropna().rank(method="min",ascending =False) 
ranks.to_csv("results/rankings.csv")

df_sensit_with_results = pd.read_csv("results/df_sensit_results.csv").set_index("country")
df_sensit = pd.read_csv("results/df_sensit_input.csv").set_index("country")


#fa_ratios for both countries and the sensitivity analysis
fa_ratios = pd.read_csv("inputs/fa_ratios.csv",index_col="country")
fa_ratios_sensit=pd.read_csv("inputs/fa_ratios_sensib.csv",index_col="country")
fa_ratios_sensit.columns = fa_ratios_sensit.columns.astype(float)

#tranforms all data to floats
fa_ratios.columns = fa_ratios.columns.astype(float)
fa_ratios=fa_ratios.astype(float)
fa_ratios;


if concat_sensit:
    #appends sensitivity anlysis
    df_with_results=df_with_results.append(df_sensit_with_results)
    fa_ratios=fa_ratios.append(fa_ratios_sensit)
    df_original = df_original.append(df_sensit)



# Computes all derivatives

In [20]:
deriv_set = np.setdiff1d( df_original.columns.values,
    ["pop","iso3","original_country","gdp_pc_pp","pov_head","avg_prod_k_ref",
     "income_elast","faref","avg_prod_k", "peref","share1_ref"
     ,"bashs","ophe","fa_ref","protectionref"])


pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)


     

In [21]:
#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")


In [22]:
#new dataframe with countries in rows and (resilience type,input) as column
#score_card_set = ["resilience","resilience_no_shock","resilience_no_shock_no_uspcale","resilience_no_shock_no_SP"] 
score_card_set = ["resilience","risk", "fa", "v_p", "v_r", "risk_to_assets", "fap", "far"]

# derivatives= compute_derivative(df_original,score_card_set,deriv_set, fa_ratios=fa_ratios, verbose_output=True)

derivatives= compute_derivative(df_original,score_card_set,deriv_set,verbose_output=True)

#derivatives_nofr= compute_derivative(df_original,score_card_set,deriv_set, verbose_output=True)
derivatives["resilience"].to_csv("results/deriv.csv")
derivatives["risk"].to_csv("results/deriv_risk.csv")



working on  v_s
ambigous sign for H
ambigous sign for T_rebuild_K
ambigous sign for pe
ambigous sign for protection
ambigous sign for pv
ambigous sign for share1
ambigous sign for v


In [23]:
if debug:
    print(derivatives.risk.protection.argmax(),derivatives.risk.protection.max())
    derivatives.risk.protection.hist()
    print(derivatives.risk.protection.sort_values(ascending=True).head())
    print(derivatives.risk.protection[["Sweden","Spain"]])
    
    derivatives.resilience.protection.hist(bins=100)
    print(derivatives.resilience.protection.sort_values(ascending=False).head(),derivatives.resilience.protection.argmax())

In [24]:

writer = make_excel_report(derivatives, "results/signs.xlsx" )
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")

In [25]:
if not concat_sensit:
    deri_sensit = compute_derivative(df_sensit.select_dtypes(exclude=[object]),score_card_set,deriv_set, fa_ratios=fa_ratios_sensit, verbose_output=False)
#derivatives_nofr= compute_derivative(df_original,score_card_set,deriv_set, verbose_output=True)


    deri_sensit["resilience"].to_csv("results/deriv_sensit.csv")
    deri_sensit["risk"].to_csv("results/deriv_risk_sensit.csv")

    writer = make_excel_report(deri_sensit, "results/signs_sensit.xlsx" )
    try :
        writer.save()
    except PermissionError:
        warnings.warn("Cannot write excel file. Check that it's not opened and try again")    

    

# Write Excel scorecards

In [26]:
if False:
    abs_derivative = signs*derivatives["resilience"] #abs value
    step_for_one = (0.01/abs_derivative).replace([-np.inf,np.inf],[np.nan,np.nan])


    headr = list(itertools.product(derivatives.dropna().index.values,['level', "ranking",'der','for_one']))
    scores=  pd.DataFrame(index=deriv_set, columns=pd.MultiIndex.from_tuples(headr))


    for c in derivatives.dropna().index:
        scores[(c,'level')]=df_original.ix[c]
        scores[(c,'der')]=abs_derivative.ix[c]
        scores[(c,'for_one')]=step_for_one.ix[c]
        scores[(c,'ranking')]=ranks.ix[df_original.ix[c,"original_country"]]


    for c in derivatives.dropna().index:
    #for c in ["France"]:
        with pd.ExcelWriter('scorecards/'+c.lower().replace(" ","_").replace("\\","")+'.xlsx', engine='xlsxwriter') as writer:
            percent = writer.book.add_format()
            percent.set_num_format("0.0%")

            outs = df_with_results.ix[df_with_results.index==c,score_card_set]
            outs.transpose().to_excel(writer)
            scores[c].reset_index().rename(columns={"index":"input"}).to_excel(writer,startrow =2+len(score_card_set),index=False)
